In [22]:
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
# from PIL import Image
import xml.etree.ElementTree as ET
# from io import BytesIO
# import dlib
# from xmltodict import parse
import _pickle as pickle
import sys
from csv import reader
from xml.dom import minidom
import math
import random

np.set_printoptions(threshold=sys.maxsize)

In [2]:
#get all annotations and image names
#extract last number
store_path = "data/data2"
images = [f for f in listdir(store_path+"/images") if isfile(join(store_path+"/images", f))]
next_num = len(images)
prefix = 'maksssksksss'

In [3]:
next_num

853

In [4]:
path_data3 = "data/data3/"
path_im3="data/data3/images/" 

In [5]:
df = pd.read_csv(path_data3+'train.csv')
data3_df = df[(df['classname'] == 'face_with_mask') | (df['classname'] == 'face_no_mask')]
data_3_group_by_name = data3_df.groupby("name")

# image3_df = data3_df.apply(lambda row: cv2.cvtColor(cv2.imread(os.path.join(path_im3,row['name'])),cv2.COLOR_BGR2RGB), axis=1)

# data3_df = data3_df.merge(image3_df.rename('image'), left_index=True, right_index=True)

In [6]:
print(data_3_group_by_name)

In [7]:
path_data1 = "data/data1/"
#get last in data2 and rename and add
path_im1_with="data/data1/with_mask/"
#get last in data2 and rename and add
path_im1_without="data/data1/without_mask/" 

In [8]:
def getBoundaryBoxPoints(img, objectList, net, label=None):
    h, w = img.shape[:2]
    blob = cv2.dnn.blobFromImage(img, 1.0,(300, 300), (104.0, 117.0, 123.0))
    net.setInput(blob)
    faces = net.forward()
    #to draw faces on image
    rectangle_list = []
    for i in range(faces.shape[2]):
            objectDict = {}
            confidence = faces[0, 0, i, 2]
            if confidence > 0.76:
                box = faces[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x, y, x1, y1) = box.astype("int")
                (startX, startY) = (max(0, x), max(0, y))
                (endX, endY) = (min(w - 1, x1), min(h - 1, y1))
                
                objectDict['name'] = filename.rsplit('/', 1)[-1]
                objectDict['classname'] = label
                objectDict['x1'] = startX
                objectDict['y1'] = startY
                objectDict['x2'] = endX
                objectDict['y2'] = endY
#                 objectDict['image'] = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                objectList.append(dict(objectDict))
    

In [9]:
#TODO: change code later 
modelFile = "models/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "models/deploy.prototxt"
net = cv2.dnn.readNetFromCaffe(path_data1+configFile, path_data1+modelFile)
objectList_with = []
objectList_without = []

for filename in os.listdir(path_im1_with):
    img = cv2.imread(path_im1_with+filename)
    if img is not None:
        getBoundaryBoxPoints(img, objectList_with, net, 'face_with_mask')
    
for filename in os.listdir(path_im1_without):
    img = cv2.imread(path_im1_without+filename)
    if img is not None:
        getBoundaryBoxPoints(img, objectList_without, net, 'face_no_mask')

In [10]:
data1_df_with = pd.DataFrame.from_records(objectList_with)
data_1_with_group_by_name = data1_df_with.groupby("name")
# data1_df.to_csv('data1.csv')

data1_df_without = pd.DataFrame.from_records(objectList_without)
data_1_without_group_by_name = data1_df_without.groupby("name")

In [11]:
def get_size(path,name):
    image = cv2.cvtColor(cv2.imread(os.path.join(path,name)),cv2.COLOR_BGR2RGB)
    return image.shape

In [12]:
def get_min_max(num1,num2):
    if num1 <= num2:
        return num1,num2
    else:
        return num2,num1

In [13]:
def get_image(path,name):
    image = cv2.imread(os.path.join(path,name))
    return image

In [14]:
def create_xml(store_path,file_path,newname,imagename,group):
    annotation = ET.Element("annotation") 
    
    folder = ET.SubElement(annotation, "folder") 
    folder.text = "images"
    filename = ET.SubElement(annotation, "filename") 
    filename.text = newname+".png"
    size = ET.SubElement(annotation, "size") 
    imagelength,imagewidth,imagechannel = get_size(file_path,imagename)
    width = ET.SubElement(size,"width")
    width.text = str(imagewidth)
    length = ET.SubElement(size,"length")
    length.text = str(imagelength)
    depth = ET.SubElement(size,"depth")
    depth.text = str(imagechannel)
    
    segmented = ET.SubElement(annotation,"segmented")
    segmented.text = '0'
        
    for i,row in group.iterrows():
        obj = ET.SubElement(annotation,"object")
        name = ET.SubElement(obj,"name")
        name.text = row['classname']
        pose = ET.SubElement(obj,"pose")
        pose.text = "none"
        truncated = ET.SubElement(obj,"truncated")
        truncated.text = "0"
        occluded = ET.SubElement(obj,"occluded")
        occluded.text = "0"
        difficult = ET.SubElement(obj,"difficult")
        difficult.text = "0"
        bndbox = ET.SubElement(obj,"bndbox")
        
        imagexmin,imagexmax = get_min_max(row['x1'],row['x2'])
        imageymin,imageymax = get_min_max(row['y1'],row['y2'])
        
        
        xmin = ET.SubElement(bndbox,"xmin")
        xmin.text = str(imagexmin)
        ymin = ET.SubElement(bndbox,"ymin")
        ymin.text = str(imageymin)
        xmax = ET.SubElement(bndbox,"xmax")
        xmax.text = str(imagexmax)
        ymax = ET.SubElement(bndbox,"ymax")
        ymax.text = str(imageymax)
        
    voc = ET.ElementTree(annotation) 
    image = get_image(file_path,imagename)
    with open(store_path+"/annotations/"+newname+".xml", "wb") as f : 
        voc.write(f) 
    
    cv2.imwrite(store_path+"/images/"+newname+'.png', image)
        

In [18]:
prefix+str(next_num)

'maksssksksss5591'

In [16]:
for name, group in data_1_with_group_by_name:
    filename = prefix+str(next_num)    
    create_xml(store_path,path_im1_with,filename,name,group) 
    next_num = next_num+1
for name, group in data_1_without_group_by_name:
    filename = prefix+str(next_num)  
    create_xml(store_path,path_im1_without,filename,name,group)
    next_num = next_num+1
for name, group in data_3_group_by_name:
    filename = prefix+str(next_num)  
    create_xml(store_path,path_data3,filename,name,group)
    next_num = next_num+1

In [17]:
store_path = "data/data2"
images = [f for f in listdir(store_path+"/images") if isfile(join(store_path+"/images", f))]
total = len(images)
total

5591

In [27]:
num_train = math.ceil(total*.9)
num_test = total - num_train
images = [f for f in listdir(store_path+"/images") if isfile(join(store_path+"/images", f))]
train = random.sample(images, num_train)
test = [image for image in images if image not in train]
print(len(train))
print(len(test))

5032
559


In [77]:
def make_new_voc(voc_file,new_image_name,output_path):
    new_voc_file = new_image_name[:-3]+'xml'
    tree = ET.parse(voc_file) 
    root = tree.getroot()
    root[1].text = new_image_name
    tree.write(output_path+new_voc_file)

In [70]:
for i,file in enumerate(train):
    new_image_name = file[:12]+str(i)+'.png'
    voc_name = file[:-3]+'xml'
    image = cv2.imread(os.path.join(store_path+"/images",file))
    voc_file = os.path.join(store_path+"/annotations",voc_name)
    make_new_voc(voc_file,new_image_name,store_path+"/random_sample/training/annotations/")
    cv2.imwrite(store_path+"/random_sample/training/images/"+new_image_name, image)

In [78]:
for i,file in enumerate(test):
    new_image_name = file[:12]+str(i+len(train))+'.png'
    voc_name = file[:-3]+'xml'
    image = cv2.imread(os.path.join(store_path+"/images",file))
    voc_file = os.path.join(store_path+"/annotations",voc_name)
    make_new_voc(voc_file,new_image_name,store_path+"/random_sample/testing/annotations/")
    cv2.imwrite(store_path+"/random_sample/testing/images/"+new_image_name, image)

In [ ]:
path_im2="data/data2/images/" 
path_xml2="data/data2/annotations/" 

In [ ]:
def getObjectProperties(filename, objectList):
    objectDict = {}
    for event_type, element in  ET.iterparse(filename, events=('start', 'end')):
        if event_type == 'start' and element.tag == 'filename':
            objectDict['name'] = element.text
        if event_type == 'end' and element.tag == 'name':
            objectDict['classname'] = element.text
        if event_type == 'end' and element.tag == 'xmin':
            objectDict['x1'] = int(element.text)
        if event_type == 'end' and element.tag == 'ymin':
            objectDict['y1'] = int(element.text)
        if event_type == 'end' and element.tag == 'xmax':
            objectDict['x2'] = int(element.text)
        if event_type == 'end' and element.tag == 'ymax':
            objectDict['y2'] = int(element.text)
        if event_type == 'end' and element.tag == 'object':
            objectList.append(dict(objectDict))
    return objectList

In [ ]:
objectList = []
for filename in os.listdir(path_xml2):
    with open(path_xml2+filename) as fd:
        getObjectProperties(fd,objectList)

In [ ]:
data2_df = pd.DataFrame.from_records(objectList)
image2_df = data2_df.apply(lambda row: cv2.cvtColor(cv2.imread(os.path.join(path_im2,row['name'])),cv2.COLOR_BGR2RGB), axis=1)
data2_df = data2_df.merge(image2_df.rename('image'), left_index=True, right_index=True)

data2_df['classname'] = data2_df['classname'].replace(['with_mask','without_mask'],['face_with_mask','face_no_mask'])
print("Generating data2.csv")
data2_df.to_csv('data2.csv')
print("Done")

print("Generating data2.pkl")
with open("data2.pkl", 'wb') as output:
    pickle.dump(data2_df, output, -1)
print("Done")

In [ ]:
#combine all csv
data1 = pd.read_csv("data1.csv",header=0)
data2 = pd.read_csv("data2.csv",header=0)
data3 = pd.read_csv("data3.csv",header=0)
merged_data = pd.concat([data1,data2,data3],ignore_index=True,sort=False)
merged_data.to_csv('all_data.csv')

In [ ]:
merged_data.isnull().values.any()